In [1]:
import os
import injection_utils as iu

In [2]:
!pwd

/global/cscratch1/sd/shl159/projects/fake_injection_summer_2022/dia_improvement/devel


# 1. Repo Preparation

Install package 
- obs_lsst: git clone https://github.com/lsst/obs_lsst.git

Checkout the correct branch:

```
cd obs_lsst
git checkout -b fake_test 30368a2b8c2941f1a04c704048c33e056a6af57b
cd ..
```
- dia_pipe: git clone https://github.com/LSSTDESC/dia_pipe.git

Checkout the correct branch:

```
cd dia_pipe
git checkout -b fake_test 8ae16a9f5e5c1e4f174626b7f1e002b9d0c69b8f
cd ..
```

```
commit 8ae16a9f5e5c1e4f174626b7f1e002b9d0c69b8f (HEAD, origin/#19)
Author: Bob Armstrong <rearmstr@gmail.com>
Date:   Tue Feb 2 14:16:07 2021 -0800
```

Create setup.sh
```
#！/bin/bash

source /cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/v20.0.0/loadLSST.bash

setup lsst_distrib

# scons only need to be done once. We cannot comment out the following three lines after the first running. setup -jr dia_pipe need to be kept.
cd dia_pipe
scons
cd ..
setup -jr dia_pipe

export HDF5_USE_FILE_LOCKING=FALSE
export OMP_NUM_THREADS=1

echo 'done'
```
In terminal, run `source setup.sh`

# 2. Inject Fakes

Create a file named `test_injection.py` with the following content. It is important to use the same version of stack packages for injection and subtraction.

```
import os
import injection_utils as iu

COADD_REPO = (
    '/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i'
    '/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-coadd-wfd-dr6-v1-grizy'
)
fake_path = (
    './injection_test'
)
os.makedirs(fake_path)
calexp_path = os.path.join(fake_path, 'calexp')
os.makedirs(calexp_path)
yaml_path = os.path.join(fake_path, 'repositoryCfg.yaml')
with open(f'{yaml_path}', "w+") as file:
    file.write("!RepositoryCfg_v1\n")
    file.write("_mapper: !!python/name:lsst.obs.lsst.imsim.ImsimMapper \'\'\n")
    file.write("_mapperArgs: {}\n")
    file.write("_parents:\n")
    file.write(f"- {COADD_REPO}\n")
    file.write("_policy: null\n")
    file.write("_root: null\n")
    file.write("dirty: true\n")
    
calexp_repo = 'injection_test'
calexp_id = {'visit': 1013665, 'detector': 79, 'filter': 'i'}
magVar = 16
coords = [(i, j) for i in range(500, 3500, 500) for j in range(500, 3500, 500)]

iu.inject_fakes_to_calexp(calexp_repo, calexp_id, magVar, coords, poisson=False)  
```

In terminal, run `python test_injection.py`.

# 3. Test subtraction

Create config `diffimConfig_al.py`, with following contents:

```
from lsst.meas.algorithms import LoadIndexedReferenceObjectsTask
config.imageDifference.refObjLoader.retarget(LoadIndexedReferenceObjectsTask)
config.imageDifference.refObjLoader.load('obs_lsst/config/filterMap.py')
config.imageDifference.kernelSourcesFromRef = True
config.ccdKey = 'detector'
```

Test subtraction without fake injection:
```
imageDifferenceDriver.py /global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-coadd-wfd-dr6-v1-grizy \
    --output test_al \
    --id visit=1013665 detector=79  -C diffimConfig_al.py \
    --clobber-config --clobber-versions --cores 4
```

Next, test subtraction with fake injection:

```
imageDifferenceDriver.py ./injection_test \
    --output test_inj \
    --id visit=1013665 detector=79  -C diffimConfig_al.py \
    --clobber-config --clobber-versions --cores 4
```